In [0]:
tiers = ["rawdata", "silver", "gold"]
adls_paths = {tier: f"abfss://{tier}@bigdatacuoiky.dfs.core.windows.net/" for tier in tiers}

bronze_adls = adls_paths["rawdata"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"] 

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

[FileInfo(path='abfss://gold@bigdatacuoiky.dfs.core.windows.net/earthquake_events_gold/', name='earthquake_events_gold/', size=0, modificationTime=1748192400000)]

In [0]:
import requests
import json
from datetime import date,timedelta

In [0]:
start_date = date.today() - timedelta(1)
end_date = date.today()

In [0]:
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

try:
    # Make the GET request to fetch data
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    data = response.json().get('features', [])

    if not data:
        print("No data returned for the specified date range.")
    else:
        # Specify the ADLS path
        file_path = f"{bronze_adls}/{start_date}_earthquake_data.json"

        # Save the JSON data
        json_data = json.dumps(data, indent=4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f"Data successfully saved to {file_path}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")

Wrote 276633 bytes.
Data successfully saved to abfss://rawdata@bigdatacuoiky.dfs.core.windows.net//2025-05-25_earthquake_data.json


In [0]:
# Define your variables
output_data = {
    "start_date": start_date.isoformat(),
    "end_date": end_date.isoformat(),
    "bronze_adls": bronze_adls,
    "silver_adls": silver_adls,
    "gold_adls": gold_adls
}

# Return the dictionary directly
dbutils.jobs.taskValues.set(key = "bronze_output", value = output_data)